In [1]:
import pandas as pd
import pymongo
import pprint
from collections import Counter
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
from pymongo import MongoClient
client = MongoClient()

In [3]:
#db = client.hkust
db = client.msbd500103

### Load Training Data

In [4]:
#trData = db.tr_data
trData = db.training_record

In [5]:
trData = trData.find({})
trData = list(trData)
df_trData = pd.DataFrame(trData)
df_trData.head()

,_id,article_contentid,article_dwelltime,article_l1_categories,article_l2_categories,article_lan,article_publisher,article_title,article_update_time,dt,...,user_country,user_gender,user_gender_weight,user_gp_frequency,user_gp_keywords,user_id,user_maxmind_city,user_maxmind_country_iso_code,user_maxmind_state_iso_code,user_model
0,5a07224d6c0c65269105e6c7,1l5c17e72og_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170111,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
1,5a07224d6c0c65269105e6c8,mQ56173a2KT_us,NaN,NaN,NaN,en,NaN,NaN,NaN,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
2,5a07224d6c0c65269105e6c9,Ly7407d4cuC_us,31.0,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988...",en,The Sun,"Dad caught selling his baby son for £9,000 to ...",1.484656e+09,20170117,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
3,5a07224d6c0c65269105e6ca,AY628126a1w_us,NaN,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]",en,Bustle,11 Weird Ways Your Relationship With Your Pare...,1.482379e+09,20170110,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710
4,5a07224d6c0c65269105e6cb,jv70536a3bt_us,NaN,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895...",en,Los Angeles Times,Ivanka Trump goes public about exiting fashion...,1.484223e+09,20170113,...,US,2.0,0.763295,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME...",NaN,614E304DE3B07EBCDD4E11382578EF5A,Glen Allen,US,VA,SPH-L710


In [165]:
df_trData = df_trData[~df_trData['user_gp_frequency'].isnull()]
df_trData = df_trData[~df_trData['article_title'].isnull()]

In [166]:
#sophia remark get the mapping of user article
df_ua_map = df_trData[['user_id', 'article_contentid']]
df_ua_map.shape

(168276, 2)

In [167]:
df_ua_map = df_ua_map.drop_duplicates(subset = ['user_id', 'article_contentid'], keep='first')
df_ua_map.shape

(139811, 2)

### Prepare User View

In [168]:
df_user = df_trData[['user_id', 'user_age', 'user_gender', 'user_brand', 'user_maxmind_city', 'user_model']]
df_user.head()

,user_id,user_age,user_gender,user_brand,user_maxmind_city,user_model
2,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,Glen Allen,SPH-L710
3,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,Glen Allen,SPH-L710
4,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,Glen Allen,SPH-L710
5,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,Glen Allen,SPH-L710
6,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,Lebanon,SAMSUNG-SM-G890A


In [169]:
df_user['user_age'] = df_user['user_age'].fillna(0)
df_user['user_gender'] = df_user['user_gender'].fillna(0)
df_user['user_brand'] = df_user['user_brand'].str.lower().fillna('na')
df_user['user_maxmind_city'] = df_user['user_maxmind_city'].str.lower().fillna('na')
df_user['user_model'] = df_user['user_model'].str.lower().fillna('na')
df_user.head()

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

,user_id,user_age,user_gender,user_brand,user_maxmind_city,user_model
2,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,glen allen,sph-l710
3,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,glen allen,sph-l710
4,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,glen allen,sph-l710
5,614E304DE3B07EBCDD4E11382578EF5A,2.0,2.0,samsung,glen allen,sph-l710
6,CD2F2B61135DB27BA3230F91466ED793,3.0,2.0,samsung,lebanon,samsung-sm-g890a


In [170]:
df_user = pd.get_dummies(df_user, columns=['user_age', 'user_gender','user_brand', 'user_maxmind_city', 'user_model'])
df_user = df_user.fillna(0)
df_user = df_user.drop_duplicates(subset=['user_id'], keep='first')
df_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_model_z955l,user_model_z958,user_model_z959,user_model_z963u,user_model_z963vl,user_model_z970,user_model_z981,user_model_z987,user_model_z988,user_model_zte_n9511
2,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
11,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
12,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [171]:
df_user.shape

(7857, 3010)

### Prepare App View

In [172]:
df_app = df_trData[['user_id', 'user_gp_frequency']]
df_app.head()

,user_id,user_gp_frequency
2,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
3,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
4,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
5,614E304DE3B07EBCDD4E11382578EF5A,"{u'SHOPPING': 1, u'BUSINESS': 1, u'ENTERTAINME..."
6,CD2F2B61135DB27BA3230F91466ED793,"{u'PRODUCTIVITY': 1, u'COMMUNICATION': 1, u'TR..."


In [173]:
df_app_category = pd.DataFrame.from_records(df_app['user_gp_frequency'].tolist())
df_app_category = df_app_category.fillna(0.1)#change from fillna(0) to fillna(0.1)
df_app_category.head()

,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,ENTERTAINMENT,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
1,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
2,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
3,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,2.0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
4,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,0.1,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1


In [174]:
df_app = df_app.merge(df_app_category, how='left', left_index=True, right_index=True)
df_app = df_app.drop(['user_gp_frequency'], axis=1)
df_app = df_app.drop_duplicates(subset=['user_id'], keep='first')
df_app.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
2,614E304DE3B07EBCDD4E11382578EF5A,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
6,CD2F2B61135DB27BA3230F91466ED793,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
11,56F27AD705D8564F7900470AA793DBD2,0.1,0.1,0.1,0.1,0.1,0.1,4.0,0.1,0.1,...,2.0,1.0,2.0,3.0,1.0,5.0,0.1,1.0,2.0,0.1
12,8EE461BA3AA2595AC91FCDD833EC1711,0.1,0.1,0.1,0.1,0.1,0.1,4.0,0.1,0.1,...,2.0,1.0,2.0,3.0,1.0,5.0,0.1,1.0,2.0,0.1
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0.1,0.1,0.1,0.1,2.0,0.1,4.0,0.1,0.1,...,0.1,0.1,1.0,3.0,0.1,10.0,0.1,3.0,7.0,2.0


In [175]:
df_app = df_app.fillna(0.1)#change from fillna(0) to fillna(0.1)

In [176]:
df_app.shape

(7857, 53)

In [177]:
def sumCat(row):
    l = row.tolist()
    #print l
    row['totalCat'] = sum(l[1:])
    return row

In [178]:
df_app1 = df_app.drop(['user_id'], axis = 1).apply(lambda row: sumCat(row), axis = 1)
df_app1 = df_app[['user_id']].merge(df_app1, how = 'left', left_index=True, right_index=True)
df_app2 = df_app1[df_app1['totalCat'] != 0].drop(['totalCat'], axis = 1)

In [179]:
df_app2.shape

(7857, 53)

In [180]:
df_app2.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
2,614E304DE3B07EBCDD4E11382578EF5A,0.1,0.1,0.1,1.0,1.0,0.1,1.0,0.1,0.1,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
6,CD2F2B61135DB27BA3230F91466ED793,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
11,56F27AD705D8564F7900470AA793DBD2,0.1,0.1,0.1,0.1,0.1,0.1,4.0,0.1,0.1,...,2.0,1.0,2.0,3.0,1.0,5.0,0.1,1.0,2.0,0.1
12,8EE461BA3AA2595AC91FCDD833EC1711,0.1,0.1,0.1,0.1,0.1,0.1,4.0,0.1,0.1,...,2.0,1.0,2.0,3.0,1.0,5.0,0.1,1.0,2.0,0.1
15,5BA72C11B1E94BA50F1E66F4B1362CE6,0.1,0.1,0.1,0.1,2.0,0.1,4.0,0.1,0.1,...,0.1,0.1,1.0,3.0,0.1,10.0,0.1,3.0,7.0,2.0


In [181]:
app_category = df_app.drop(['user_id'], axis = 1).columns
app_category

Index([u'ART_AND_DESIGN', u'AUTO_AND_VEHICLES', u'BEAUTY',
       u'BOOKS_AND_REFERENCE', u'BUSINESS', u'COMICS', u'COMMUNICATION',
       u'DATING', u'EDUCATION', u'ENTERTAINMENT', u'EVENTS', u'FAMILY_ACTION',
       u'FINANCE', u'FOOD_AND_DRINK', u'GAME_ACTION', u'GAME_ADVENTURE',
       u'GAME_ARCADE', u'GAME_BOARD', u'GAME_CARD', u'GAME_CASINO',
       u'GAME_CASUAL', u'GAME_EDUCATIONAL', u'GAME_MUSIC', u'GAME_PUZZLE',
       u'GAME_RACING', u'GAME_ROLE_PLAYING', u'GAME_SIMULATION',
       u'GAME_SPORTS', u'GAME_STRATEGY', u'GAME_TRIVIA', u'GAME_WORD',
       u'HEALTH_AND_FITNESS', u'HOUSE_AND_HOME', u'LIBRARIES_AND_DEMO',
       u'LIFESTYLE', u'MAPS_AND_NAVIGATION', u'MEDIA_AND_VIDEO', u'MEDICAL',
       u'MUSIC_AND_AUDIO', u'NEWS_AND_MAGAZINES', u'PARENTING',
       u'PERSONALIZATION', u'PHOTOGRAPHY', u'PRODUCTIVITY', u'SHOPPING',
       u'SOCIAL', u'SPORTS', u'TOOLS', u'TRANSPORTATION', u'TRAVEL_AND_LOCAL',
       u'VIDEO_PLAYERS', u'WEATHER'],
      dtype='object')

## Merge user and app

In [182]:
df_user_app = df_user.merge(df_app2, how = 'left', on = 'user_id')
#df_user_app = df_user_app.drop(['totalCat'], axis = 1)
df_user_app = df_user_app.fillna(0.1) #change from fillna(0) to fillna(0.1)
df_user_app.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
1,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
2,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.1,1.0,2.0,0.1
3,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,2.0,1.0,2.0,3.0,1.0,5.0,0.1,1.0,2.0,0.1
4,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0.1,0.1,1.0,3.0,0.1,10.0,0.1,3.0,7.0,2.0


In [183]:
a = df_user_app.drop(['user_id'], axis = 1).as_matrix()
samples = list(a)
type(samples[0])

from sklearn.neighbors import NearestNeighbors
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(samples) 

#get the nearest user index - use for loop later
#print(neigh.kneighbors([samples[20]])) [1][0][0]

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

## Get user related category as label

get a table to map user_id with read article and the l1_categories and l2_categories

In [184]:
df_user_app_label = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories']]#sophia remark : should add article_update_time as a feature
df_user_app_label['article_l1_categories'] = df_user_app_label['article_l1_categories'].fillna({})
df_user_app_label['article_l2_categories'] = df_user_app_label['article_l2_categories'].fillna({})
df_user_app_label.head()

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,article_contentid,article_l1_categories,article_l2_categories
2,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,"[{u'name': u'1000780', u'weight': 0.4546442536...","[{u'name': u'1000667', u'weight': 0.6867789988..."
3,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,"[{u'name': u'1000288', u'weight': 0.9748942874...","[{u'name': u'1000288', u'weight': 0.44672383556}]"
4,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,"[{u'name': u'1000031', u'weight': 0.4396004700...","[{u'name': u'1000069', u'weight': 0.8568055895..."
5,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,"[{u'name': u'1000031', u'weight': 0.8209544133...","[{u'name': u'1000069', u'weight': 0.4637899765..."
6,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,"[{u'name': u'1001091', u'weight': 0.4180664718...","[{u'name': u'1001091', u'weight': 0.4180664718..."


Extract l1_category and l2_category from the list

In [185]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [186]:
df_user_app_label['l1_cat'] = df_user_app_label.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_user_app_label['l2_cat'] = df_user_app_label.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
#df_user_app_label['l1_cat_w'] = df_user_app_label.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
#df_user_app_label['l2_cat_w'] = df_user_app_label.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_user_app_label = df_user_app_label.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_user_app_label.head()

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,user_id,article_contentid,l1_cat,l2_cat
2,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,1000780,1000667
3,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,1000288,1000288
4,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,1000031,1000069
5,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,1000031,1000069
6,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,1001091,1001091


In [187]:
def sumCat(l):
    l = l.unique()
    return l.tolist()

In [188]:
df_user_cat = df_user_app_label.groupby(['user_id']).apply(lambda row: sumCat(row['l2_cat']))
df_user_cat = pd.DataFrame(df_user_cat).reset_index()
df_user_cat = df_user_cat.rename(columns={0:'l2_cat'})
df_user_cat.head()

,user_id,l2_cat
0,00108FC5F1F3031ADD536548124A4912,[1000069]
1,00310455B7B24D5CDEC892351B8D1D51,"[1000667, 1000611, 1000780, 1000395]"
2,00381B68ACD6CC83635CDDD9B40F0BAF,"[1000671, 1000680, 1000681]"
3,003B2215B070CDB50C0819C72E208F0A,"[1000395, 1000290, 1000288, 1000069, 1000611]"
4,003B8D414199B9C5EB7CF8C94BCC6787,[1000069]


In [189]:
def splitCat(row):
    for i in row['l2_cat']:
        if i is not None:
            row['user_l2_cat' + i] = 1
    return row

In [190]:
df_user_cat = df_user_cat.apply(lambda row:splitCat(row), axis = 1)
df_user_cat.head()

,l2_cat,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,[1000069],00108FC5F1F3031ADD536548124A4912,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[1000667, 1000611, 1000780, 1000395]",00310455B7B24D5CDEC892351B8D1D51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[1000671, 1000680, 1000681]",00381B68ACD6CC83635CDDD9B40F0BAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[1000395, 1000290, 1000288, 1000069, 1000611]",003B2215B070CDB50C0819C72E208F0A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,[1000069],003B8D414199B9C5EB7CF8C94BCC6787,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [191]:
df_user_cat = df_user_cat.fillna(0.1).drop(['l2_cat'],axis = 1)#change from fillna(0) to fillna(0.1)
df_user_cat.head()

,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,user_l2_cat10000177,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,00108FC5F1F3031ADD536548124A4912,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,00310455B7B24D5CDEC892351B8D1D51,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,00381B68ACD6CC83635CDDD9B40F0BAF,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,003B2215B070CDB50C0819C72E208F0A,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,003B8D414199B9C5EB7CF8C94BCC6787,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [192]:
df_user_app_cat = df_user_app.merge(df_user_cat, how = 'left', on = 'user_id')
df_user_app_cat.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,614E304DE3B07EBCDD4E11382578EF5A,0,0,1,0,0,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,CD2F2B61135DB27BA3230F91466ED793,0,0,0,1,0,0,0,1,0,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1
2,56F27AD705D8564F7900470AA793DBD2,0,0,0,0,1,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,8EE461BA3AA2595AC91FCDD833EC1711,0,0,1,0,0,0,0,1,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,5BA72C11B1E94BA50F1E66F4B1362CE6,0,0,1,0,0,0,1,0,0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


### Prepare Article View

In [193]:
df_article = df_trData[['user_id', 'article_contentid', 'article_l1_categories', 'article_l2_categories', 'article_title', 'article_update_time']]#sophia remark : should add article_update_time as a feature
df_article = df_article.sort_values(by=['article_title'], ascending=False)
df_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_title,article_update_time
73696,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,1.485441e+09
125132,A4F138114721E738AD8C2768C11450A9,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,1.485388e+09
45894,986759E2FD77D818BEDC17BFF6376900,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,1.485441e+09
136028,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1.482883e+09
136080,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,1.482883e+09


In [194]:
from sklearn import preprocessing
#Create x, where x the 'scores' column's values as floats
x = df_article['article_update_time'].values.astype(float)
x = x.reshape(-1,1)
# Create a minimum and maximum processor object
min_max_scaler = preprocessing.MinMaxScaler()

# Create an object to transform the data to fit minmax processor
x_scaled = min_max_scaler.fit_transform(x)

# Run the normalizer on the dataframe
df_article['article_update_time'] = x_scaled

In [195]:
df_article = df_article.drop_duplicates(subset=['article_contentid'], keep='first')
df_article = df_article.reset_index(drop = True)
df_article.head()

,user_id,article_contentid,article_l1_categories,article_l2_categories,article_title,article_update_time
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,"[{u'name': u'1000780', u'weight': 0.8786688469...","[{u'name': u'1000667', u'weight': 0.4648142188...",﻿Vulnerable teen 'livestreams suicide' on Face...,0.950863
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,"[{u'name': u'1000780', u'weight': 0.0405336841...","[{u'name': u'1000780', u'weight': 0.0405336841...",【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.342006
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,"[{u'name': u'8000001', u'weight': 0.999}]","[{u'name': u'8000001', u'weight': 0.999}]",♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.597340
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,"[{u'name': u'1000931', u'weight': 0.1268057078...","[{u'name': u'1000931', u'weight': 0.1268057078...",▶[Future Bass] ★ MRVLZ - On The Inside,0.923283
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,"[{u'name': u'1000780', u'weight': 0.4859073658...","[{u'name': u'1000780', u'weight': 0.4859073658...",€1m in funding sought for new air ambulance se...,0.949038


In [196]:
def getDetail(cat, field):
    if (str(cat) != 'nan'):
        if (len(cat) > 0):
            return cat[0][field]
    else:
        return '0'

In [197]:
df_article['l1_cat'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'name'), axis=1)
df_article['l2_cat'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'name'), axis=1)
df_article['l1_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l1_categories'],'weight'), axis=1)
df_article['l2_cat_w'] = df_article.apply(lambda row: getDetail(row['article_l2_categories'],'weight'), axis=1)
df_article = df_article.drop(['article_l1_categories', 'article_l2_categories'], axis=1)
df_article.head()

,user_id,article_contentid,article_title,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,﻿Vulnerable teen 'livestreams suicide' on Face...,0.950863,1000780,1000667,0.878669,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,【E-girls】ちび丸子ちゃん―踊りを学びましょう～,0.342006,1000780,1000780,0.040534,0.040534
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,♠ Funny Videos ♠ Best FAILS ♠ December 2016 Co...,0.597340,8000001,8000001,0.999000,0.999000
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,▶[Future Bass] ★ MRVLZ - On The Inside,0.923283,1000931,1000931,0.126806,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,€1m in funding sought for new air ambulance se...,0.949038,1000780,1000780,0.485907,0.485907


In [198]:
df_article.shape

(49149, 8)

### Clean title

In [199]:
#df_article = df_article[~df_article['article_title'].isnull()]
#df_article.head()

In [200]:
#df_article = df_article.drop(['article_title','clean_article_title'], axis=1)
#Sophia commend : the following line is to be run if title is not included
df_article = df_article.drop(['article_title'], axis=1)
df_article.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907


In [201]:
#df_article = pd.get_dummies(df_article, columns=['article_publisher'])
df_article = df_article.fillna(0)
df_article.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907


In [202]:
df_cat = df_article[['l1_cat', 'l2_cat', 'l1_cat_w', 'l2_cat_w']]
df_cat = pd.get_dummies(df_cat, columns=['l1_cat', 'l2_cat'])
df_cat = df_cat.fillna(0.1)#change from fillna(0) to fillna(0.1)
#df_cat = df_cat.drop(['l1_cat_0', 'l2_cat_0'], axis=1)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.040534,0.040534,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.999000,0.999000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [203]:
df_cat = df_cat.merge(df_article[['l1_cat', 'l2_cat']], how='left', left_index=True, right_index=True)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000667
1,0.040534,0.040534,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780
2,0.999000,0.999000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,8000001,8000001
3,0.126806,0.126806,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000931,1000931
4,0.485907,0.485907,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [204]:
def updateCatWeight(row):
    cat = row['l1_cat']
    row['l1_cat_'+str(cat)] = row['l1_cat_w']
    return row

In [205]:
df_cat = df_cat.apply(lambda row: updateCatWeight(row), axis=1)
df_cat.head()

,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,...,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,l1_cat,l2_cat
0,0.878669,0.464814,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000780,1000667
1,0.040534,0.040534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000780,1000780
2,0.999000,0.999000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,1,8000001,8000001
3,0.126806,0.126806,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000931,1000931
4,0.485907,0.485907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1000780,1000780


In [206]:
df_cat = df_cat.drop(['l1_cat','l2_cat','l1_cat_w', 'l2_cat_w'], axis=1)
df_cat.head()

,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,l1_cat_1000298,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
df_article = df_article.merge(df_cat, how='left', left_index=True, right_index=True)
#sophia remark: commend here
#df_article = df_article.drop(['article_publisher','article_title'], axis = 1)
df_article.head()

,user_id,article_contentid,article_update_time,l1_cat,l2_cat,l1_cat_w,l2_cat_w,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,7C62CF4B33EF7303881DB654E68FF85E,v77c577e1qo_us,0.950863,1000780,1000667,0.878669,0.464814,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,17A177FDE5C3C9F500D784FF8D985AD7,0i65cb6eaNb_us,0.342006,1000780,1000780,0.040534,0.040534,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,DB7995E79922E8A96E1585DA8C6EFB51,ag639e0eeDK_us,0.597340,8000001,8000001,0.999000,0.999000,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
3,84E7779E484630375F5F0F6175045B4B,7m799a2c4mX_us,0.923283,1000931,1000931,0.126806,0.126806,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,DE83F55126D386CF852A76F25FE0FE04,hQ7cfd994Tv_us,0.949038,1000780,1000780,0.485907,0.485907,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [208]:
df_article_2 = df_article.drop(['l1_cat','l2_cat','l1_cat_w','l2_cat_w'], axis = 1)
df_article_2.shape

(49149, 186)

In [209]:
#df_user.head()

Change by sophia --> should keep all user-article pair

In [ ]:
#df_user_article = df_ua_map.merge(df_user, how='left', on='user_id')
#df_user_article.head()

In [ ]:
#df_user_article = df_user_article.merge(df_article.drop(['user_id'],axis = 1), how = 'left', on='article_contentid')
#df_user_article.head()

In [ ]:
#df_user_article.shape

In [ ]:
#df_user_article = df_user_article.fillna(0)
#df_user_article = df_user_article[(df_user_article['l1_cat'] != 0) & (df_user_article['l2_cat'] != 0)]
#df_user_article.head()

In [ ]:
#df_ua_user = df_user_article[df_user.columns]
##df_ua_user = df_ua_user.drop(['user_id'], axis=1)
#df_ua_user = df_ua_user.fillna(0)
#df_ua_user.head()

In [ ]:
#df_ua_article = df_user_article[df_article.columns]
#df_ua_article = df_ua_article.drop(['user_id', 'article_contentid'], axis=1)
##df_ua_article = df_ua_article.drop(['article_contentid'], axis=1)
#df_ua_article = df_ua_article.fillna(0)
#df_ua_article.head()

In [ ]:
#Sophia remark: save current df_app : user article or others contain ascii - cannot save
#df_user.to_csv('testdata1127/df_user.csv', sep='\t', encoding='utf-8')
#df_ua_article.to_csv('testdata1127/df_ua_article.csv', sep='\t', encoding='utf-8')
#df_article.to_csv('testdata1127/df_article.csv', sep='\t', encoding='utf-8')
#df_app.to_csv('testdata1127/df_app.csv')
#df_app2.to_csv('testdata1127/df_app2.csv')

### Prepare Train Test Set

In [210]:
import random
train_ratio = 0.9
num_train_user = int(round(len(df_user)*train_ratio))
train_user_id = random.sample(df_user['user_id'].tolist(), num_train_user)

In [211]:
len(train_user_id)

7071

In [212]:
test_user_id = df_user[~df_user['user_id'].isin(train_user_id)]['user_id'].tolist()
len(test_user_id)

786

In [213]:
old_new_ratio = 0.8
num_old_user = int(round(len(test_user_id)*train_ratio))
old_user_id = random.sample(test_user_id, num_old_user)
len(old_user_id)

707

In [214]:
new_user_id = [i for i in test_user_id if i not in old_user_id]
len(new_user_id)

79

In [215]:
final_train_user_id = []
final_train_user_id.extend(train_user_id)
final_train_user_id.extend(old_user_id)
len(final_train_user_id)

7778

In [216]:
final_test_user_id = []
final_test_user_id.extend(new_user_id)
final_test_user_id.extend(old_user_id)
len(final_test_user_id)

786

KNN - in final test, there is no need to get final_train and final_test(previous steps)

In [217]:
df_train_user_0 = df_user_app[df_user_app['user_id'].isin(final_train_user_id)].fillna(0.1)
df_test_user_0 = df_user_app[df_user_app['user_id'].isin(final_test_user_id)].fillna(0.1)

In [218]:
df_train_user_0.shape, df_test_user_0.shape

((7778, 3062), (786, 3062))

In [219]:
a = df_train_user_0.drop(['user_id'], axis = 1).as_matrix()
samples = list(a)
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(samples) 

#get the nearest user index - use for loop later
#print(neigh.kneighbors([samples[15]])) [1][0]

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=1, p=2, radius=1.0)

In [220]:
def findNeighbor(row):
    l = row.values
    row['neibor'] = neigh.kneighbors([l]) [1][0][0]
    return row

In [234]:
# user with neighbor information w/o user id
df_test_user_1 = df_test_user_0.drop(['user_id'],axis = 1).apply(lambda row: findNeighbor(row), axis = 1)#drop(['user_id'],axis = 1)
# user with neighbor information and user id
df_test_user_3 = df_test_user_0[['user_id']].merge(df_test_user_1, how = 'left', left_index = True, right_index = True)
df_test_user_3.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER,neibor
9,292BB97BB1606137F47D15CD95613681,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,5.0,0.1,0.1,1.0,0.1,9
16,FDA822F54256CAE5209A7FD11B87503F,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,3.0,0.1,0.1,0.1,3.0,0.1,0.1,1.0,0.1,16
22,3FD27317628111BD18212E3FFD828FB6,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,22
28,63DCD9FACDA032C8045C92D4954AD5E8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,3.0,0.1,3.0,0.1,22.0,0.1,2.0,1.0,0.1,28
37,D8419484B8B08621C786F79AE86C71C2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,37


In [235]:
df_user_cat.head()

,user_id,user_l2_cat0010000000,user_l2_cat1000001,user_l2_cat1000002,user_l2_cat1000007,user_l2_cat10000101,user_l2_cat1000012,user_l2_cat10000162,user_l2_cat1000017,user_l2_cat10000177,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,00108FC5F1F3031ADD536548124A4912,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,00310455B7B24D5CDEC892351B8D1D51,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,00381B68ACD6CC83635CDDD9B40F0BAF,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,003B2215B070CDB50C0819C72E208F0A,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,003B8D414199B9C5EB7CF8C94BCC6787,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [238]:
df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')
df_train_user.shape

(7778, 3217)

In [239]:
art_cat_col = [i for i in df_train_user.columns if i not in df_train_user_0.columns]
len(art_cat_col)

155

In [262]:
user_fea_col = [i for i in df_train_user.columns \
                if i not in app_category and 'user_brand' not in i\
                and 'user_model' not in i  and 'article_contentid' != i]
len(user_fea_col)

2470

In [263]:
df_test_user = df_test_user_3.merge(df_train_user[art_cat_col], how = 'left', left_on='neibor', right_index=True)
df_test_user = df_test_user.drop(['neibor'],axis = 1)
df_test_user.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_brand_acer,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
9,292BB97BB1606137F47D15CD95613681,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
16,FDA822F54256CAE5209A7FD11B87503F,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1
22,3FD27317628111BD18212E3FFD828FB6,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
28,63DCD9FACDA032C8045C92D4954AD5E8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
37,D8419484B8B08621C786F79AE86C71C2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [264]:
df_test_user.shape, df_train_user.shape

((786, 3217), (7778, 3217))

In [276]:
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
df_test_user_2 = df_test_user[user_fea_col].fillna(0)
#df_test_user_2 = df_test_user.fillna(0)
#df_test_user_arr = df_test_user_2.values
#df_test_user_arr.shape

In [279]:
df_test_user_2.columns

Index([                u'user_id',            u'user_age_0.0',
                  u'user_age_1.0',            u'user_age_2.0',
                  u'user_age_3.0',            u'user_age_4.0',
               u'user_gender_0.0',         u'user_gender_1.0',
               u'user_gender_2.0', u'user_maxmind_city_'aiea',
       ...
            u'user_l2_cat1001040',      u'user_l2_cat1001049',
            u'user_l2_cat1001060',      u'user_l2_cat1001091',
            u'user_l2_cat1001103',      u'user_l2_cat1001110',
            u'user_l2_cat1001117',      u'user_l2_cat2000000',
            u'user_l2_cat6000007',      u'user_l2_cat8000001'],
      dtype='object', length=2470)

In [280]:
app_category

Index([u'ART_AND_DESIGN', u'AUTO_AND_VEHICLES', u'BEAUTY',
       u'BOOKS_AND_REFERENCE', u'BUSINESS', u'COMICS', u'COMMUNICATION',
       u'DATING', u'EDUCATION', u'ENTERTAINMENT', u'EVENTS', u'FAMILY_ACTION',
       u'FINANCE', u'FOOD_AND_DRINK', u'GAME_ACTION', u'GAME_ADVENTURE',
       u'GAME_ARCADE', u'GAME_BOARD', u'GAME_CARD', u'GAME_CASINO',
       u'GAME_CASUAL', u'GAME_EDUCATIONAL', u'GAME_MUSIC', u'GAME_PUZZLE',
       u'GAME_RACING', u'GAME_ROLE_PLAYING', u'GAME_SIMULATION',
       u'GAME_SPORTS', u'GAME_STRATEGY', u'GAME_TRIVIA', u'GAME_WORD',
       u'HEALTH_AND_FITNESS', u'HOUSE_AND_HOME', u'LIBRARIES_AND_DEMO',
       u'LIFESTYLE', u'MAPS_AND_NAVIGATION', u'MEDIA_AND_VIDEO', u'MEDICAL',
       u'MUSIC_AND_AUDIO', u'NEWS_AND_MAGAZINES', u'PARENTING',
       u'PERSONALIZATION', u'PHOTOGRAPHY', u'PRODUCTIVITY', u'SHOPPING',
       u'SOCIAL', u'SPORTS', u'TOOLS', u'TRANSPORTATION', u'TRAVEL_AND_LOCAL',
       u'VIDEO_PLAYERS', u'WEATHER'],
      dtype='object')

In [281]:
df_test_app0 = df_app2[df_app2['user_id'].isin(final_test_user_id)]
df_test_app_2 = df_test_user_0[['user_id']].merge(df_test_app0, how = 'left', on = 'user_id')
df_test_app_2.head()

,user_id,ART_AND_DESIGN,AUTO_AND_VEHICLES,BEAUTY,BOOKS_AND_REFERENCE,BUSINESS,COMICS,COMMUNICATION,DATING,EDUCATION,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,292BB97BB1606137F47D15CD95613681,0.1,0.1,0.1,0.1,0.1,0.1,1.0,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,5.0,0.1,0.1,1.0,0.1
1,FDA822F54256CAE5209A7FD11B87503F,0.1,0.1,0.1,0.1,0.1,0.1,3.0,0.1,0.1,...,0.1,3.0,0.1,0.1,0.1,3.0,0.1,0.1,1.0,0.1
2,3FD27317628111BD18212E3FFD828FB6,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,63DCD9FACDA032C8045C92D4954AD5E8,0.1,0.1,0.1,0.1,0.1,0.1,7.0,0.1,0.1,...,0.1,3.0,0.1,3.0,0.1,22.0,0.1,2.0,1.0,0.1
4,D8419484B8B08621C786F79AE86C71C2,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [282]:
df_test_app_2 = df_test_app_2.fillna(0.1)
df_test_user_arr = df_test_app_2.values
df_test_user_arr.shape 

(786, 53)

In [283]:
df_test_user_2 = df_test_user_2.merge(df_test_app_2, how = 'left', on = 'user_id')
df_test_user_2 = df_test_user_2.drop(['user_id'],axis = 1)
df_test_user_arr = df_test_user_2.values
df_test_user_arr.shape

(786, 2521)

In [284]:
len(df_ua_map)

139811

In [84]:
#df_train_user = df_train_user_0.merge(df_user_cat, how = 'left', on = 'user_id')

In [289]:
all_user_id = train_user_id + old_user_id
df_final_train_user_article = df_ua_map[(df_ua_map['user_id'].isin(all_user_id))]
df_final_train_user_article = df_final_train_user_article.merge(df_train_user[user_fea_col], how = 'left', on = 'user_id')

In [290]:
print (df_final_train_user_article.shape)
df_final_train_user_article.head()

(138636, 2471)


,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,0,0,1,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
2,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,0,0,1,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1


In [291]:
df_final_train_user_article = df_final_train_user_article.merge(df_article_2.drop(['user_id'],axis = 1), how = 'left', on = 'article_contentid')
df_final_train_user_article.head()

,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,614E304DE3B07EBCDD4E11382578EF5A,AY628126a1w_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,614E304DE3B07EBCDD4E11382578EF5A,1V70d401fdc_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [292]:
df_final_train_user_article.shape

(138636, 2655)

In [293]:
df_final_train_user_article = df_final_train_user_article.iloc[[i for i in df_final_train_user_article.index if i%2 == 0]].reset_index(drop=True)
df_final_train_user_article.head()

,user_id,article_contentid,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,614E304DE3B07EBCDD4E11382578EF5A,Ly7407d4cuC_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,614E304DE3B07EBCDD4E11382578EF5A,jv70536a3bt_us,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,CD2F2B61135DB27BA3230F91466ED793,jQ6c581bbr2_us,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
3,CD2F2B61135DB27BA3230F91466ED793,7Y7152e79c5_us,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,CD2F2B61135DB27BA3230F91466ED793,pf6fc9375ah_us,0,0,0,1,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [294]:
#df_old_user_article = df_ua_map[df_ua_map['user_id'].isin(old_user_id)]
#df_old_user_article = df_old_user_article.merge(df_train_user[test_col], how = 'left', on = 'user_id')
#df_old_user_article = df_old_user_article.drop_duplicates(subset=['user_id', 'article_contentid'], keep='first').reset_index(drop=True)
#df_old_user_article = df_old_user_article.sort_values('user_id').reset_index(drop=True)
#df_old_user_article = df_old_user_article.merge(df_article.drop(['user_id'],axis = 1), how = 'left', on = 'article_contentid')
#df_old_user_article.head()

In [295]:
#df_final_train = df_final_train_user_article.merge(df_user, how='left', on='user_id')
df_final_train = df_final_train_user_article.fillna(0.1)
#sophia remark: add app data
#df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')

In [296]:
df_final_train.shape

(69318, 2655)

In [297]:
df_final_train = df_final_train.merge(df_app2, how='left', on='user_id')
###sophia remark, check wether there is NAN or all 0 here
#df_final_train = df_final_train.dropna()
#df_final_train.head()

In [298]:
df_final_train.shape

(69318, 2707)

In [299]:
user_feature = [i for i in df_final_train.columns if 'user_' in i and i != 'user_id']
user_feature

['user_age_0.0',
 'user_age_1.0',
 'user_age_2.0',
 'user_age_3.0',
 'user_age_4.0',
 'user_gender_0.0',
 'user_gender_1.0',
 'user_gender_2.0',
 u"user_maxmind_city_'aiea",
 u"user_maxmind_city_'ewa beach",
 u'user_maxmind_city_abbeville',
 u'user_maxmind_city_aberdeen',
 u'user_maxmind_city_abilene',
 u'user_maxmind_city_abingdon',
 u'user_maxmind_city_absecon',
 u'user_maxmind_city_acworth',
 u'user_maxmind_city_ada',
 u'user_maxmind_city_adamsville',
 u'user_maxmind_city_adrian',
 u'user_maxmind_city_ahmedabad',
 u'user_maxmind_city_aiken',
 u'user_maxmind_city_akron',
 u'user_maxmind_city_alabaster',
 u'user_maxmind_city_alameda',
 u'user_maxmind_city_alamogordo',
 u'user_maxmind_city_albany',
 u'user_maxmind_city_albemarle',
 u'user_maxmind_city_albertville',
 u'user_maxmind_city_albuquerque',
 u'user_maxmind_city_alden',
 u'user_maxmind_city_aledo',
 u'user_maxmind_city_alexandria',
 u'user_maxmind_city_algoma',
 u'user_maxmind_city_alhambra',
 u'user_maxmind_city_alice',
 u'use

In [345]:
article_feature = [i for i in df_final_train.columns if 'user_' not in i and i not in ['article_contentid',
 'article_title',
 'article_publisher',
 'l1_cat',
 'l2_cat',
 'l1_cat_w',
 'l2_cat_w',
 'l1_cat_w_x',
 'l2_cat_w_x',
 'l1_cat_w_y',
 'l2_cat_w_y',
 'totalCat'] and i not in app_category]
#article_feature = 
article_feature

[u'article_update_time',
 'l1_cat_0',
 u'l1_cat_0010000000',
 u'l1_cat_1000001',
 u'l1_cat_1000031',
 u'l1_cat_1000044',
 u'l1_cat_1000111',
 u'l1_cat_1000123',
 u'l1_cat_1000267',
 u'l1_cat_1000288',
 u'l1_cat_1000298',
 u'l1_cat_1000374',
 u'l1_cat_1000395',
 u'l1_cat_1000560',
 u'l1_cat_1000620',
 u'l1_cat_1000637',
 u'l1_cat_1000661',
 u'l1_cat_1000721',
 u'l1_cat_1000742',
 u'l1_cat_1000780',
 u'l1_cat_1000849',
 u'l1_cat_1000931',
 u'l1_cat_1000992',
 u'l1_cat_1001039',
 u'l1_cat_1001091',
 u'l1_cat_2000000',
 u'l1_cat_6000007',
 u'l1_cat_8000001',
 'l2_cat_0',
 u'l2_cat_0010000000',
 u'l2_cat_1000001',
 u'l2_cat_1000002',
 u'l2_cat_1000007',
 u'l2_cat_10000101',
 u'l2_cat_1000012',
 u'l2_cat_10000162',
 u'l2_cat_1000017',
 u'l2_cat_10000177',
 u'l2_cat_10000179',
 u'l2_cat_1000025',
 u'l2_cat_1000028',
 u'l2_cat_1000031',
 u'l2_cat_1000032',
 u'l2_cat_1000042',
 u'l2_cat_1000044',
 u'l2_cat_1000050',
 u'l2_cat_10000667',
 u'l2_cat_1000069',
 u'l2_cat_1000072',
 u'l2_cat_1000073'

In [346]:
uapp_feature = [i for i in df_test_user_2.columns if i not in article_feature]
len(uapp_feature)

2521

In [347]:
df_final_train.shape

(69318, 2707)

In [348]:
df_train_user= df_final_train[uapp_feature]
df_train_user.head()
#df_train_app = df_final_train[app_category]
#df_train_app.head()

,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_maxmind_city_'aiea,user_maxmind_city_'ewa beach,...,PHOTOGRAPHY,PRODUCTIVITY,SHOPPING,SOCIAL,SPORTS,TOOLS,TRANSPORTATION,TRAVEL_AND_LOCAL,VIDEO_PLAYERS,WEATHER
0,0,0,1,0,0,0,0,1,0,0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
1,0,0,1,0,0,0,0,1,0,0,...,0.1,3.0,1.0,1.0,0.1,5.0,0.1,0.1,2.0,0.1
2,0,0,0,1,0,0,0,1,0,0,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
3,0,0,0,1,0,0,0,1,0,0,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1
4,0,0,0,1,0,0,0,1,0,0,...,2.0,1.0,0.1,1.0,0.1,3.0,0.1,1.0,0.1,0.1


In [349]:
df_train_article = df_final_train[article_feature]
df_train_article.head()

,article_update_time,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001040,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001
0,0.764028,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0.660763,0.0,0.0,0.0,0.4396,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0.828354,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0
3,0.688781,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0.672347,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0,0,0,0,0,0


In [350]:
df_train_user.shape

(69318, 2521)

In [351]:
df_train_article.shape

(69318, 184)

In [352]:
#df_final_train[df_final_train['user_id'] == 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7']

In [353]:
#df_article[df_article['user_id'] == 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7']

In [354]:
df_test_article = df_article.drop_duplicates(subset=['article_contentid'], keep='first')[article_feature]
df_test_article.shape

(49149, 184)

In [355]:
len(article_feature)

184

In [356]:
article_feature.append(u'article_contentid')
len(article_feature)

185

In [358]:
df_test_article0 = df_article.drop_duplicates(subset=['article_contentid'], keep='first')[article_feature]
df_test_article0.head()

,article_update_time,l1_cat_0,l1_cat_0010000000,l1_cat_1000001,l1_cat_1000031,l1_cat_1000044,l1_cat_1000111,l1_cat_1000123,l1_cat_1000267,l1_cat_1000288,...,l2_cat_1001049,l2_cat_1001060,l2_cat_1001091,l2_cat_1001103,l2_cat_1001110,l2_cat_1001117,l2_cat_2000000,l2_cat_6000007,l2_cat_8000001,article_contentid
0,0.950863,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,v77c577e1qo_us
1,0.342006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0i65cb6eaNb_us
2,0.597340,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,1,ag639e0eeDK_us
3,0.923283,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,7m799a2c4mX_us
4,0.949038,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,hQ7cfd994Tv_us


In [359]:
df_test_article_arr1 = df_test_article.values

In [360]:
df_test_article_arr1.shape

(49149, 184)

In [361]:
#df_test_user1 = df_user[df_user['user_id']== 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7']

In [362]:
#df_test_user1.head()

In [363]:
#df_test_user = df_user[df_user['user_id'] == 'F4CCDDAA1BAE16B0AF3BCB82801A3CF7'][user_feature]
#user_array = [df_test_user.values[0] for i in range(0, len(df_test_article))]
#df_test_user = np.asarray(user_array)
#df_test_user.shape

In [364]:
#df_test_app = df_app2[df_app2['user_id']=='F4CCDDAA1BAE16B0AF3BCB82801A3CF7'][app_category]
#app_array = [df_test_app.values[0] for i in range(0, len(df_test_article))]
#df_test_app = np.asarray(app_array)
#df_test_app.shape

In [365]:
#final_test_user_id

In [366]:
y_train = [1 for i in range(len(df_train_user.values))]
len(y_train)

69318

### Build Model By Keras (User Vs App Vs Article)

In [367]:
df_train_user.columns

Index([                u'user_age_0.0',                 u'user_age_1.0',
                       u'user_age_2.0',                 u'user_age_3.0',
                       u'user_age_4.0',              u'user_gender_0.0',
                    u'user_gender_1.0',              u'user_gender_2.0',
            u'user_maxmind_city_'aiea', u'user_maxmind_city_'ewa beach',
       ...
                        u'PHOTOGRAPHY',                 u'PRODUCTIVITY',
                           u'SHOPPING',                       u'SOCIAL',
                             u'SPORTS',                        u'TOOLS',
                     u'TRANSPORTATION',             u'TRAVEL_AND_LOCAL',
                      u'VIDEO_PLAYERS',                      u'WEATHER'],
      dtype='object', length=2521)

In [368]:
df_train_user.shape

(69318, 2521)

In [377]:
#user_fea_woid = [i for i in user_fea_col if i != 'user_id']
#len(user_fea_woid)

In [378]:
df_train_user_arr = df_train_user.values
#df_train_app_arr = df_train_app.values

In [379]:
df_train_user_arr.shape

(69318, 2521)

In [380]:
df_train_article.shape

(69318, 184)

In [381]:
df_train_article_arr = df_train_article.values
df_train_article_arr.shape

(69318, 184)

In [382]:
y_train = [1 for i in range(len(df_train_user))]
len(y_train)

69318

In [383]:
from keras.models import Sequential, Model
from keras.layers import Dense, Merge, Activation, Dropout, Reshape

Using TensorFlow backend.


In [384]:
# create user model
userModel1 = Sequential()
userModel1.add(Dense(500, input_shape=(2521,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(50, input_shape=(500,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(30, input_shape=(50,)))
userModel1.add(Activation('tanh'))
userModel1.add(Dropout(0.4))
userModel1.add(Dense(10, input_shape=(30,)))
userModel1.add(Activation('tanh'))
userModel1.add(Reshape((10,1)))

In [386]:
# create article model
articleModel = Sequential()
#articleModel.add(Dense(2000, input_shape=(8064,)))
#articleModel.add(Activation('tanh'))
#articleModel.add(Dropout(0.4))
#articleModel.add(Dense(500, input_shape=(2000,)))
#articleModel.add(Activation('tanh'))
#articleModel.add(Dropout(0.4))
articleModel.add(Dense(50, input_shape=(184,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(30, input_shape=(50,)))
articleModel.add(Activation('tanh'))
articleModel.add(Dropout(0.4))
articleModel.add(Dense(10, input_shape=(30,)))
articleModel.add(Activation('tanh'))
articleModel.add(Reshape((10,1)))

In [387]:
model1 = Sequential()
model1.add(Merge([userModel1,articleModel], mode='cos', dot_axes=2, output_shape=(1,)))
model1.add(Reshape((1,)))

In [388]:
# Compile model 1
model1.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

# Compile model 2
#model2.compile(loss='cosine_proximity', optimizer='adam', metrics=['accuracy','precision','recall'])
#model2.fit([df_train_app_arr ,df_train_article_arr] ,y_train, nb_epoch=1, batch_size=32)

Epoch 1/1
69318/69318 [==============================] - 49s - loss: -0.9292 - acc: 0.7629 - precision: 0.9977 - recall: 0.7629    


In [389]:
model1.fit([df_train_user_arr ,df_train_article_arr] ,y_train, nb_epoch=4, batch_size=32)

Epoch 1/4
69318/69318 [==============================] - 45s - loss: -0.9997 - acc: 0.9798 - precision: 1.0000 - recall: 0.9798    
Epoch 2/4
69318/69318 [==============================] - 45s - loss: -0.9999 - acc: 0.9955 - precision: 1.0000 - recall: 0.9955    
Epoch 3/4
69318/69318 [==============================] - 45s - loss: -1.0000 - acc: 0.9985 - precision: 1.0000 - recall: 0.9985    
Epoch 4/4
69318/69318 [==============================] - 47s - loss: -1.0000 - acc: 0.9990 - precision: 1.0000 - recall: 0.9990    


In [ ]:
#y_pred = model.predict([df_test_user,df_test_article_arr1, df_test_app], batch_size=32, verbose=0)
#y_pred_1 = np.hstack(y_pred)
#np.argsort(y_pred_1)[:100]

### Calculate precision and recall

In [391]:
#sophia remark get the mapping of user article
#df_ua_map = df_trData[['user_id', 'article_contentid']]

#Sophia remark: get test user list
#df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_user[df_user['user_id'].isin(final_test_user_id)]
#df_test_user = df_test_user.drop(['user_id'], axis=1)
#df_test_user = df_test_user.fillna(0)
df_test_user_arr1 = df_test_user.values
#df_test_user_arr1.shape

#Sophia remark: get test app list
df_test_app0 = df_app2[df_app2['user_id'].isin(final_test_user_id)]
df_test_app = df_user[df_user['user_id'].isin(final_test_user_id)].merge(df_test_app0, how = 'left', on = 'user_id')[app_category]
df_test_app = df_test_app.fillna(0)
df_test_app_arr1 = df_test_app.values
#df_test_app_arr1.shape

#sophia remark: get article list
df_test_article0 = df_article.drop_duplicates(subset=['article_contentid'], keep='first')#[article_feature]
#df_test_article0.head()

df_test_article_arr1 = df_test_article.values

In [392]:
t_u = df_test_user[user_fea_col]
t_u = t_u.reset_index(drop=True)
t_u.head()

,user_id,user_age_0.0,user_age_1.0,user_age_2.0,user_age_3.0,user_age_4.0,user_gender_0.0,user_gender_1.0,user_gender_2.0,user_maxmind_city_'aiea,...,user_l2_cat1001040,user_l2_cat1001049,user_l2_cat1001060,user_l2_cat1001091,user_l2_cat1001103,user_l2_cat1001110,user_l2_cat1001117,user_l2_cat2000000,user_l2_cat6000007,user_l2_cat8000001
0,292BB97BB1606137F47D15CD95613681,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
1,FDA822F54256CAE5209A7FD11B87503F,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1
2,3FD27317628111BD18212E3FFD828FB6,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
3,63DCD9FACDA032C8045C92D4954AD5E8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
4,D8419484B8B08621C786F79AE86C71C2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [393]:
df_test_user0 = df_user[df_user['user_id'].isin(final_test_user_id)]

In [394]:
df_test_article_arr1[0:5]

array([[ 0.95086286,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.87866885,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0. 

In [397]:
l = len(df_test_article_arr1)
uid = df_test_user0['user_id'].tolist()[0]
user = np.array([df_test_user_arr[0]]*l)
#user
#app = np.array([df_test_app_arr[0]]*l)
#apply user and article array to do predict
up = pd.DataFrame(model1.predict([user,df_test_article_arr1], batch_size=32))
up.head()

,0
0,0.928337
1,0.922698
2,0.933316
3,0.944861
4,0.932589


In [400]:
l = len(df_test_article_arr1)
precision = list()
recall = list()

for idx, user in enumerate(df_test_user_arr):
    #if idx > 5:
    #    break;
    #for each user-article pair: get the user id and user info
    uid = df_test_user0['user_id'].tolist()[idx]
    user = np.array([df_test_user_arr[idx]]*l)
    #app = np.array([df_test_app_arr1[idx]]*l)
    #apply user and article array to do predict
    up = pd.DataFrame(model1.predict([user,df_test_article_arr1], batch_size=32))
    df_test_article1=df_test_article0.reset_index(drop = True)\
                            .merge(up, how = 'left', left_index=True, right_index=True)\
                            .rename(columns={0:'u0_score'})[['article_contentid', 'u0_score']]
    df_test_article1 = df_test_article1.sort_values(by='u0_score', ascending=False)
    df_test_article1['y_pred'] = 0
    df_test_article1[:5]['y_pred']=1
    df_match = df_ua_map[df_ua_map['user_id'] == uid][['article_contentid']]
    df_match['y_true'] = 1
    df_test_article1 = df_test_article1.merge(df_match, how = 'left', on = 'article_contentid')\
                                       .fillna(0)
    tp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 1)].shape[0]
    fp = df_test_article1[(df_test_article1['y_pred'] == 1) & (df_test_article1['y_true'] == 0)].shape[0]
    fn = df_test_article1[(df_test_article1['y_pred'] == 0) & (df_test_article1['y_true'] == 1)].shape[0]

    p = tp/(float(tp+fp))
    r = tp/(float(tp+fn))
    print (idx, uid, p,r)
    if p != 0:
        precision.append(p)
    if r != 0:
        recall.append(r)
    
print (precision, recall)

/Users/sophia.gao/.pyenv/versions/miniconda2-latest/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0, u'292BB97BB1606137F47D15CD95613681', 0.0, 0.0)
(1, u'FDA822F54256CAE5209A7FD11B87503F', 0.0, 0.0)
(2, u'3FD27317628111BD18212E3FFD828FB6', 0.0, 0.0)
(3, u'63DCD9FACDA032C8045C92D4954AD5E8', 0.0, 0.0)
(4, u'D8419484B8B08621C786F79AE86C71C2', 0.0, 0.0)
(5, u'E016D48A88152F9F137DF9DDF290C39F', 0.0, 0.0)
(6, u'40C3F37008AF3DAF878D0A8691B8258E', 0.0, 0.0)
(7, u'459B96B1F70D26383BBF2F63C86D6BB7', 0.0, 0.0)
(8, u'D79CC4620095EAF4E21BA529129E8107', 0.0, 0.0)
(9, u'CDE7637996E630E1FA3C327A934E8339', 0.0, 0.0)
(10, u'B86935B368FE8941F00512E292A623ED', 0.0, 0.0)
(11, u'90E9D5579C993D73E310FF7A3D6AE62C', 0.0, 0.0)
(12, u'EDC2B8045E3F2D9FBD71E5736E516ABC', 0.0, 0.0)
(13, u'D89143FBB6B1395FC4628E27832333F2', 0.0, 0.0)
(14, u'2F01D4766DF9DD9089EC9B2030086756', 0.0, 0.0)
(15, u'AFAE834F8AC28FA7CF1B4C39D8E463A3', 0.0, 0.0)
(16, u'1B99F512223524991B480674CA914AF0', 0.0, 0.0)
(17, u'40929E249813D17DE0BE52F61CDDCD62', 0.0, 0.0)
(18, u'CC96BA15EF74E888344DFC7A32AB77EB', 0.0, 0.0)
(19, u'E300FB58DA32E2C

(157, u'F7A4A4FE797C52448D2DCD9237E47535', 0.0, 0.0)
(158, u'F3D40E188C12492BD1BD03FCCFA57A3E', 0.0, 0.0)
(159, u'A23F9D004EDFFE2820D90A39D1087720', 0.0, 0.0)
(160, u'069934F48B98BD12F73E7128E408C75B', 0.0, 0.0)
(161, u'11A5E5196BA7259BD887386C77771C54', 0.0, 0.0)
(162, u'85D82F0DDBD3E09FF7BBDBE8F13CFD32', 0.0, 0.0)
(163, u'6F6F02A8885887000D5DA326DEAE337B', 0.0, 0.0)
(164, u'5319E0E74C30B6F1272C63256E73754C', 0.0, 0.0)
(165, u'A2D315C3565DAAAC3B8A070C853EAE8B', 0.0, 0.0)
(166, u'834C39D362DD5EE38AEAC0E0763B82D5', 0.0, 0.0)
(167, u'A96441D5404D9A19ADD86865159DCDD3', 0.0, 0.0)
(168, u'7E501BE70260B824205E4D86FA10CC57', 0.0, 0.0)
(169, u'6EF863D3797E6C219D198AA1B8EA24C6', 0.0, 0.0)
(170, u'C054B4A46137ED6531764D9FE9F82532', 0.0, 0.0)
(171, u'081267417FFB9E4B991518E189F22F3C', 0.0, 0.0)
(172, u'0394F5B6684FD1524604512C634D7AD1', 0.0, 0.0)
(173, u'BF1ABA69CE3708307664084FCD2D8A9E', 0.0, 0.0)
(174, u'37C31B4CDF130979873B3AD41F6DD875', 0.0, 0.0)
(175, u'AC02BB3BC7E55C095B7E06CC4F674BC1', 0.0

(312, u'E83B07D14C25FACD3C10EA5CA13E0D40', 0.0, 0.0)
(313, u'3B2B8CD526DB74390C2E466740BDAAA0', 0.0, 0.0)
(314, u'84C92AE6968CB8E58275EAD337ED387B', 0.0, 0.0)
(315, u'EDD7347405B26FADEE01F95CBC8C58CF', 0.0, 0.0)
(316, u'72BE783E32C775F42790EB1FE73EC360', 0.0, 0.0)
(317, u'12F51E66A64BB6AF266D4370589D21DA', 0.0, 0.0)
(318, u'F9FA9D6A63402A8F6259E726AF67573A', 0.0, 0.0)
(319, u'AB3628F6E69F70FAF0EED0D99BFBD9AF', 0.0, 0.0)
(320, u'6CBFDFC866375375402E80753287D71D', 0.0, 0.0)
(321, u'4FE5C805E0907B0EBFBD266B792C9FA7', 0.0, 0.0)
(322, u'BB3A94A2A05988033148A532695733E4', 0.0, 0.0)
(323, u'30A3718D572B778B277920D4FDD256B2', 0.0, 0.0)
(324, u'61C3C25013B394FEE01BD2A2502A6A38', 0.0, 0.0)
(325, u'8621DA6B506BDACE5D8B94C75F67AA33', 0.0, 0.0)
(326, u'D1A0117311241E731C10A8E08C35DFC2', 0.0, 0.0)
(327, u'E0C66F25C090C2FDA018F7C9EBAE7D00', 0.0, 0.0)
(328, u'DC3EFD43B5BE1952F5A000BB502A6D13', 0.0, 0.0)
(329, u'AFB0A46F7C27D785DF7EC908EA3AC52B', 0.0, 0.0)
(330, u'78525AB3CA9BA0A6A4BA4010BFB9CBB0', 0.0

(467, u'861AF189FFF1B079CC817D2160512A53', 0.0, 0.0)
(468, u'588410BC3165EB470F5ACC67ADD65B95', 0.0, 0.0)
(469, u'EB93A0C0A8E6E3D472C554A1ACB9E436', 0.0, 0.0)
(470, u'575101C6B248E62B5FBDCADE62955FB2', 0.0, 0.0)
(471, u'F4809E03CA20E395C6335078019C7A24', 0.0, 0.0)
(472, u'C255FEEE2E7AE4D9D72D72E4919482C3', 0.0, 0.0)
(473, u'6EA9DDCC778E0DF0931F33D6BD29C6B6', 0.0, 0.0)
(474, u'D584FCDB83E76D67B7A83B9529A4F739', 0.0, 0.0)
(475, u'019A3785381C82296CADFFFE2B6CD276', 0.0, 0.0)
(476, u'15BA907A744AD69600BCEA2D74DA7C81', 0.0, 0.0)
(477, u'5C753EDC916BD1B26C8CD3F8949D1A32', 0.0, 0.0)
(478, u'837E4CD48D7E54E3ED271631286E50CC', 0.0, 0.0)
(479, u'D618946CB53B057151F15F482013FD79', 0.0, 0.0)
(480, u'05FF1EF7811F7F1A4EEB186C851A9132', 0.0, 0.0)
(481, u'411727798B93ED273A8E62135FA222DB', 0.0, 0.0)
(482, u'90B2E7082A2E555A894B232471B5C80F', 0.0, 0.0)
(483, u'D9E4B40C0579E20C784B94710EA50BC8', 0.0, 0.0)
(484, u'4ED760E07D5EB2C9D672A48234962051', 0.0, 0.0)
(485, u'69B640384237B9F242930FF346877F20', 0.0

(622, u'9BBDC5F6443725885212B584C97500AE', 0.0, 0.0)
(623, u'9593BA87E1A8746CB890F3BF7330DEC8', 0.0, 0.0)
(624, u'68123806C68C66E761CD3F43C7F01DB9', 0.0, 0.0)
(625, u'5BA1ED07943FE0FD65A47E5115F7DE35', 0.0, 0.0)
(626, u'BDDAB83276285D0B8245519B6CFA09A0', 0.0, 0.0)
(627, u'42945F876C600F7AD4D7AC218EEEABCF', 0.0, 0.0)
(628, u'0DDEDEC7506F9F3C047CB7C1BBCE3C13', 0.0, 0.0)
(629, u'34F67C37773DA584BC8B58A0BCA36CDE', 0.0, 0.0)
(630, u'8B47DF3927FF7C948CE369F3677A116B', 0.0, 0.0)
(631, u'4E715FA27EAD2A916045258BD4CDA543', 0.0, 0.0)
(632, u'E3F30AF8DB00E7BBB341C47C3CA26537', 0.0, 0.0)
(633, u'7A7A030373280B253041C1FC37ABD73A', 0.0, 0.0)
(634, u'0BB93031A2D49AFC043888589EBBAC1F', 0.0, 0.0)
(635, u'1255E0A0C9104CA5A9C510061B7AB810', 0.0, 0.0)
(636, u'B3394EC38C042BEAD0BA62AC0231C085', 0.0, 0.0)
(637, u'D41190CD69A45E991B54A82DD6846C99', 0.0, 0.0)
(638, u'E1725C5017AF3B9E5754539A15EAEAB8', 0.0, 0.0)
(639, u'7C376320795FA9ED7846C6261E8F61A4', 0.0, 0.0)
(640, u'8AFB5222CF8414393BBFDA98A07DC6F1', 0.0

(777, u'DC231D4C6532FC8B869A4C5AF3B48F27', 0.0, 0.0)
(778, u'036093EA6EE9533D7AE4343D7BA523AD', 0.0, 0.0)
(779, u'72D42FE3C0CD15D8295155B1E6922E4D', 0.0, 0.0)
(780, u'A6426747B3BC92DB126C8AC27A5917A8', 0.0, 0.0)
(781, u'F1CFB13269FC90DB311131B71F82E776', 0.0, 0.0)
(782, u'325F5220F5264120CC54495230FFA8ED', 0.0, 0.0)
(783, u'2AA2847282A745B356261E8D43E783E4', 0.0, 0.0)
(784, u'00E1ED486DCC8EC45179B6A0D705A7AA', 0.0, 0.0)
(785, u'7C5146AFFC0A433D39341E0E073E68B1', 0.0, 0.0)
([0.2], [0.3333333333333333])


In [ ]:
df_test_article1.head()

In [ ]:
df_test_article1[df_test_article1['article_contentid'].isin(list(df_trData[df_trData['user_id'] == 'B3B51127D5FB5B39E4273BE65CACA28C']['article_contentid']))]

In [ ]:
df_ua_map[df_ua_map['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']

In [ ]:
list(df_trData[df_trData['user_id'] == '5BA72C11B1E94BA50F1E66F4B1362CE6']['article_contentid'])

In [ ]:
t_u.head()

In [ ]:
t_u['index'] = t_u.index
t_u_group = t_u.groupby(by=[i for i in t_u.columns if i != 'index']).agg({'index':'nunique'}).reset_index(drop=True)
t_u_group = t_u_group.sort_values(by=['index'], ascending=False)
t_u_group.head()